The problem is that our major technology client is trying to figure out the best move for their business. They either want to create a base in Toronto or New York City, they want us to use the foursquare api to help them make a decision.

To solve this problem, We will be using data from Toronto including neighborhoods, boroughs, and what businesses are nearby. We will also be using data from New York City including neighborhoods, boroughs, and what businesses are nearby. by comparing the concentrations of neighborhoods, we can help the client know where to put their base to reach the greatest amount of people.

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    certifi-2018.11.29         |        py36_1000         145 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    ca-certificates-2018.11.29 |       ha4d7672_0         143 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [2]:
CLIENT_ID = 'FZZCES4SP1L1YFERINY3QU23JHSL13LB0QODW4WYSXPYUXSZ' # your Foursquare ID
CLIENT_SECRET = 'JR35DLCEC51TESBMZBVGCA5E5ZUHOB0DWAOQDAH3AHREPHUB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZZCES4SP1L1YFERINY3QU23JHSL13LB0QODW4WYSXPYUXSZ
CLIENT_SECRET:JR35DLCEC51TESBMZBVGCA5E5ZUHOB0DWAOQDAH3AHREPHUB


In [22]:
address = 'New York, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


40.7308619 -73.9871558


In [23]:
search_query = 'Technology'
radius = 500
print(search_query + ' .... OK!')

Technology .... OK!


In [24]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FZZCES4SP1L1YFERINY3QU23JHSL13LB0QODW4WYSXPYUXSZ&client_secret=JR35DLCEC51TESBMZBVGCA5E5ZUHOB0DWAOQDAH3AHREPHUB&ll=40.7308619,-73.9871558&v=20180604&query=Technology&radius=500&limit=30'

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c02f9c36a607137c3faee3a'},
 'response': {'venues': [{'id': '4cdab522a1bba35dd04890e5',
    'name': 'Foothold Technology',
    'location': {'address': '36 E 12th St Fl 5',
     'crossStreet': 'at University Pl.',
     'lat': 40.73350476361521,
     'lng': -73.99218904304851,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.73350476361521,
       'lng': -73.99218904304851}],
     'distance': 516,
     'postalCode': '10003',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['36 E 12th St Fl 5 (at University Pl.)',
      'New York, NY 10003',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d124941735',
      'name': 'Office',
      'pluralName': 'Offices',
      'shortName': 'Office',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1543698883',


In [26]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4cdab522a1bba35dd04890e5,36 E 12th St Fl 5,US,New York,United States,at University Pl.,516,"[36 E 12th St Fl 5 (at University Pl.), New Yo...","[{'label': 'display', 'lat': 40.73350476361521...",40.733505,-73.992189,10003,NY,Foothold Technology,v-1543698883
1,[],False,52f607c3498edfa025e4b93d,sheraton,US,Cairo,United States,NaN,264,"[sheraton, Cairo, NY, United States]","[{'label': 'display', 'lat': 40.7332361117005,...",40.733236,-73.987356,NaN,NY,"Arab academy for sience , technology. And Mart...",v-1543698883
2,[],False,4e035006d164b56ad5338e21,60 Washington Sq S,US,New York,United States,NaN,450,"[60 Washington Sq S, New York, NY 10012, Unite...","[{'label': 'display', 'lat': 40.7335452, 'lng'...",40.733545,-73.991160,10012,NY,Foothold technology expo,v-1543698883
3,"[{'id': '4bf58dd8d48988d1a5941735', 'name': 'C...",False,5074566fe4b0f199474d4c24,14 Washington Places,US,New York,United States,NaN,553,"[14 Washington Places, New York, NY 10003, Uni...","[{'label': 'display', 'lat': 40.728982, 'lng':...",40.728982,-73.993225,10003,NY,New York University Information Technology Aca...,v-1543698883
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,514b1c6fe4b08739959d3cc4,1 Union Sq W,US,New York,United States,14th Street,604,"[1 Union Sq W (14th Street), New York, NY 1000...","[{'label': 'display', 'lat': 40.73527849819009...",40.735278,-73.991322,10003,NY,FineLine Technology,v-1543698883


In [27]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Foothold Technology,Office,36 E 12th St Fl 5,US,New York,United States,at University Pl.,516,"[36 E 12th St Fl 5 (at University Pl.), New Yo...","[{'label': 'display', 'lat': 40.73350476361521...",40.733505,-73.992189,10003,NY,4cdab522a1bba35dd04890e5
1,"Arab academy for sience , technology. And Mart...",None,sheraton,US,Cairo,United States,NaN,264,"[sheraton, Cairo, NY, United States]","[{'label': 'display', 'lat': 40.7332361117005,...",40.733236,-73.987356,NaN,NY,52f607c3498edfa025e4b93d
2,Foothold technology expo,None,60 Washington Sq S,US,New York,United States,NaN,450,"[60 Washington Sq S, New York, NY 10012, Unite...","[{'label': 'display', 'lat': 40.7335452, 'lng'...",40.733545,-73.991160,10012,NY,4e035006d164b56ad5338e21
3,New York University Information Technology Aca...,College Lab,14 Washington Places,US,New York,United States,NaN,553,"[14 Washington Places, New York, NY 10003, Uni...","[{'label': 'display', 'lat': 40.728982, 'lng':...",40.728982,-73.993225,10003,NY,5074566fe4b0f199474d4c24
4,FineLine Technology,Office,1 Union Sq W,US,New York,United States,14th Street,604,"[1 Union Sq W (14th Street), New York, NY 1000...","[{'label': 'display', 'lat': 40.73527849819009...",40.735278,-73.991322,10003,NY,514b1c6fe4b08739959d3cc4


In [28]:
df_database=pd.read_csv("NY_zip_database.csv")
df_database

,Zipcode,City,State,Latitude,Longitude
0,11597,115 CRM FIRMS,NY,40.75,-73.58
1,11594,115 FIRMS,NY,40.75,-73.58
2,11535,ABMPS,NY,40.72,-73.64
3,11750,ABMPS,NY,40.83,-73.37
4,12404,ACCORD,NY,41.80,-74.23
5,12405,ACRA,NY,42.33,-74.09
6,13605,ADAMS,NY,43.80,-76.02
7,14410,ADAMS BASIN,NY,43.19,-77.86
8,13606,ADAMS CENTER,NY,43.85,-75.99
9,10579,ADAMS CORNERS,NY,41.34,-73.87


In [50]:
df_database=pd.read_csv("NY_zip_database.csv")
limit = 3000
df_database = df_database.iloc[0:limit, :]
df_database

,Zipcode,City,State,Latitude,Longitude
0,11597,115 CRM FIRMS,NY,40.75,-73.58
1,11594,115 FIRMS,NY,40.75,-73.58
2,11535,ABMPS,NY,40.72,-73.64
3,11750,ABMPS,NY,40.83,-73.37
4,12404,ACCORD,NY,41.80,-74.23
5,12405,ACRA,NY,42.33,-74.09
6,13605,ADAMS,NY,43.80,-76.02
7,14410,ADAMS BASIN,NY,43.19,-77.86
8,13606,ADAMS CENTER,NY,43.85,-75.99
9,10579,ADAMS CORNERS,NY,41.34,-73.87


In [51]:
NY_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(NY_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(NY_map)
Geo = folium.map.FeatureGroup()
# loop through the longitude and latitude and add each to the geo feature group
for lat, lng, in zip(df_database.Latitude, df_database.Longitude):
    Geo.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=10, # define how big you want the circle markers to be
            color='purple',
            fill=True,
            fill_color='black',
            fill_opacity=0.6
        )
    )

# add incidents to map
NY_map.add_child(Geo)
# display map

NY_map

In [20]:
address = 'Toronto, CN'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


43.6425637 -79.3870871832047


In [12]:
search_query = 'Technology'
radius = 500
print(search_query + ' .... OK!')

Technology .... OK!


In [13]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FZZCES4SP1L1YFERINY3QU23JHSL13LB0QODW4WYSXPYUXSZ&client_secret=JR35DLCEC51TESBMZBVGCA5E5ZUHOB0DWAOQDAH3AHREPHUB&ll=43.6425637,-79.3870871832047&v=20180604&query=Technology&radius=500&limit=30'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c02e5306a607137bc88244a'},
 'response': {'venues': [{'id': '5b31f46a67e5f2002c88e5ba',
    'name': 'Digital Display Advertising Media | Signage Technology Company | Marketing Ads',
    'location': {'address': '156 Front Street West, Second Floor Toronto, ON,',
     'lat': 43.64638600677007,
     'lng': -79.38419222831726,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64638600677007,
       'lng': -79.38419222831726}],
     'distance': 485,
     'postalCode': 'M5J 2L7',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['156 Front Street West, Second Floor Toronto, ON,',
      'Toronto ON M5J 2L7',
      'Canada']},
    'categories': [{'id': '56aa371be4b08b9a8d573517',
      'name': 'Business Center',
      'pluralName': 'Business Centers',
      'shortName': 'Business Center',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/default_',
       'suf

In [15]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '56aa371be4b08b9a8d573517', 'name': 'B...",False,5b31f46a67e5f2002c88e5ba,"156 Front Street West, Second Floor Toronto, ON,",CA,Toronto,Canada,485,"[156 Front Street West, Second Floor Toronto, ...","[{'label': 'display', 'lat': 43.64638600677007...",43.646386,-79.384192,M5J 2L7,ON,Digital Display Advertising Media | Signage Te...,v-1543693616
1,"[{'id': '4bf58dd8d48988d125941735', 'name': 'T...",False,55f7f2fd498efde71622c239,10 Capreol Court,CA,Toronto,Canada,583,"[10 Capreol Court, Toronto ON M5V 4B3, Canada]","[{'label': 'display', 'lat': 43.64088917701412...",43.640889,-79.393945,M5V 4B3,ON,Taksa Technology Solutions,v-1543693616


In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Digital Display Advertising Media | Signage Te...,Business Center,"156 Front Street West, Second Floor Toronto, ON,",CA,Toronto,Canada,485,"[156 Front Street West, Second Floor Toronto, ...","[{'label': 'display', 'lat': 43.64638600677007...",43.646386,-79.384192,M5J 2L7,ON,5b31f46a67e5f2002c88e5ba
1,Taksa Technology Solutions,Tech Startup,10 Capreol Court,CA,Toronto,Canada,583,"[10 Capreol Court, Toronto ON M5V 4B3, Canada]","[{'label': 'display', 'lat': 43.64088917701412...",43.640889,-79.393945,M5V 4B3,ON,55f7f2fd498efde71622c239


In [17]:
dataframe_filtered.name

0    Digital Display Advertising Media | Signage Te...
1                           Taksa Technology Solutions
Name: name, dtype: object

In [18]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
df_combined=pd.read_csv("Canada_combined.csv")
geo = folium.map.FeatureGroup()
# loop through the longitude and latitude and add each to the geo feature group
for lat, lng, in zip(df_combined.Latitude, df_combined.Longitude):
    geo.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=10, # define how big you want the circle markers to be
            color='purple',
            fill=True,
            fill_color='black',
            fill_opacity=0.6
        )
    )

# add incidents to map
venues_map.add_child(geo)
# display map
venues_map